In [ ]:
# default_exp train

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#hide 
# %load_ext autoreload 
# %autoreload 2
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

/content/drive/MyDrive/Coding/ModelAssistedLabel


# Training Models
> wrapping `yolov5/train.py`

We're building towards generating a model with a single call.

`Trainer` is a fairly bare-bones wrapper built around `train.py`.

`AutoWeights` is a more robust wrapper and allows for custom naming/placement of the results folder.

In [ ]:
# export
from ModelAssistedLabel.config import Defaults
import os

class Trainer():
  """A wrapper for Ultralytic's `test.py`
  
  Write the backbone of the model to file and then run YOLOv5's train file."""

  def __init__(self, name, yaml_file = "models/custom_yolov5s.yaml"):
    """
    sets the current directory to the project's root as defined in Defaults.

    Args:
      name: identifier for results
      yaml_file: path to write the file 
    """
    os.chdir(Defaults().root)
    self.yaml_file = yaml_file
    self.name = name
    self.template = Defaults().trainer_template

  def write_yaml(self):
    """
    Records YOLOv5 architecture
    """
    yaml = f"yolov5/{self.yaml_file}"
    if os.path.exists(yaml):
      os.remove(yaml)
    f = open(yaml,"w")
    f.writelines(self.template)
    f.close()

  def train(self, epochs):
    """
    wrapper for train.py.

    Args:
      epochs: number of iterations
    """
    self.write_yaml()
    os.chdir("yolov5")
    os.system("pip install -r requirements.txt")
    os.system(f"python train.py --img 416 --batch 16 --epochs {epochs} --data '../data.yaml' --cfg '{self.yaml_file}' --weights '' --name '{self.name}'  --cache")
    os.chdir("..")

In [ ]:
# export
from ModelAssistedLabel.train import Trainer
from ModelAssistedLabel.fileManagement import Generation
from datetime import datetime

class AutoWeights():
  """Given a bag of images (.jpg) and labels (.txt) in YOLOv5 format in a repository,
  initialize the ROOT directory with a train-valid-test split and a file needed 
  by the Ultralytics repository. Pairs are identified via having a common filename.

  Then call `generate_weights` to run `train.py`. The resultant file will try to 
  be moved to the `out_dir` and if a conflict exists, a new name will be made.
  """
  def __init__(self, name="AutoWeight <name>", out_dir=".", MAX_SIZE=5, custom_split=None, custom_data_yaml=None, verbose=True, train_path = "yolov5/runs/train"):
    """
    Args:
      out_dir: where the results of train.py are moved
      MAX_SIZE: parameter for `Generation`
      custom_data_yaml: see `Defaults`'s `data_yaml` for the default value
      verbose: Print summary information
      train_path: path to Ultralytic's default output folder
    """
    self.resource_paths = ["test/", "train/", "valid/"]
    self.name = name
    self.out_dir = out_dir
    self.train_path = train_path
    if custom_data_yaml is None:
      custom_data_yaml = Defaults().data_yaml
    self.custom_data_yaml = custom_data_yaml
    self.verbose = verbose
    self.custom_split=custom_split
    self.MAX_SIZE = MAX_SIZE

    found = []
    for r in self.resource_paths:
      if os.path.exists(r):
        found.append(r)
    if len(found) > 0:
      print("found resources:", found)
  
  def traverse_resources(self):
    for r in self.resource_paths:
      if os.path.exists(r):
        listdir = len(os.listdir(r))
        for subdir in os.listdir(r):
          print('Directory:', os.path.join(r, subdir), "|" , len(os.listdir(os.path.join(r, subdir))),"files")
      else:
        listdir = "n/a"
      print('Directory:', r, "|" , str(listdir),"files")

  def generate_weights(self, epochs, tidy_weights=True):
    """
    Creates a `Trainer` object and trains for a given amount of time.

    Args:
      epochs: number of iterations (according to docs, over 3000 is not uncommon)
      tidy_weights: if True, remove all of the resources in `self.resources`
    
    Returns:
      path to the output folder of train.py
    """
    t = Trainer(self.name)
    ldir = lambda path: set(os.listdir(path))

    before = ldir(self.train_path)
    t.train(epochs)
    after = ldir(self.train_path)
    
    assert len(after) == len(before)+1 #only should have made one new file
    diff = list(after - before)[0]

    results_path = os.path.join(self.train_path, diff)

    if tidy_weights:
      results_path = self.__tidy_weights__(results_path = results_path)

    self.__cleanup__()
    self.last_results_path = results_path
    return results_path

  def __prepare_split__(self, data_yaml, verbose, override):
    """
    Gets the local filesystem ready to run the wrapper for "train.py".

    Args:
      data_yaml:
      verbose: print summary information for the split
    """  
    if data_yaml is None:
      data_yaml = Defaults().data_yaml
    
    if override:
      self.g.set_split(MAX_SIZE=self.MAX_SIZE)
      zipped = self.g.write_split_to_disk(self.name) #create a zip file
      self.__split_and_organize_folders__(zipped=zipped)
    else:
      self.g.set_split_from_disk()
  
  def initialize_images_from_zip(self, zipped):
    """
    Assume zip file is of the following structure:
      * data.yaml
      * train/
        - images/
        - labels/
      * valid/
        - images/
        - labels/
      * test/
        - images/
        - labels/

    Extract these 4 resources to the ROOT directory and remove the original 
    part of the file.

    Args:
      verbose: print summary information about the split
    """
    assert os.path.exists(zipped)
    os.system(f'unzip "{zipped}"') #grab data
    folder = zipped[:-4] #remove the ".zip from the filename

    #move the contents of the zip file into postion within the ROOT directory
    for content in os.listdir(folder):
      os.system(f"mv '{os.path.join(folder, content)}' .")
      if os.path.isfile(os.path.join(folder, content)):
        os.system(f"mv '{content} ./yolov5/{content}")

    #removed the folder that was taken out of the zip
    os.system(f"rm -f -r '{folder}'")

  def initialize_images_from_bag(self, bag_of_images_and_labels):
    g = Generation(repo=bag_of_images_and_labels, 
                  out_dir=self.out_dir,
                  data_yaml=self.custom_data_yaml,
                  verbose=self.verbose)
    g.set_split(split_ratio=self.custom_split, MAX_SIZE=self.MAX_SIZE)  
    g.get_split()
    zipped = g.write_split_to_disk(descriptor=self.name)
    self.initialize_images_from_zip(zipped)
    os.system(f'rm -f -r "{zipped}"')
    self.g = g

  def __cleanup__(self):
    """
    Removes all resources in `self.resource_paths` from the filesystem.
    """
    for r in self.resource_paths:
      if os.path.exists(r):
        print('Removing: ', r)
        os.system(f"rm -f -r {r}")

  def __tidy_weights__(self, results_path):
    """
    Moves the results to a desired directly while ensuring that no data is overwritten

    Args:
      results_path: path to the folder that has desired information
    
    Returns:
      Path to the newly-moved results
    """      
    default_name = os.path.join(self.out_dir, os.path.basename(results_path))
    out = Defaults._itername(f"{default_name} - ", "")

    os.system(f"mv '{results_path}' '{out}'")
    return out

In [ ]:
aw = AutoWeights(out_dir="ipynb_tests/02_train_datadump", MAX_SIZE=10)
aw.__cleanup__()

reading defaults from: ModelAssistedLabel config.json


In [ ]:
aw.initialize_images_from_bag(bag_of_images_and_labels = "./Image Repo/labeled/Final Roboflow Export (841)")

call `set_split` before `write_files_to_disk`
(1) copying: ./Image Repo/labeled/Final Roboflow Export (841)/images/digittake-153-jpg_jpg.rf.c540375397ac13ca361faf4acc756a30.jpg
(2) copying: ./Image Repo/labeled/Final Roboflow Export (841)/labels/digittake-153-jpg_jpg.rf.c540375397ac13ca361faf4acc756a30.txt
(3) copying: ./Image Repo/labeled/Final Roboflow Export (841)/images/digittake-344-jpg_jpg.rf.cbe8fd13bf132dbb26a2ba0611d29c8f.jpg
(4) copying: ./Image Repo/labeled/Final Roboflow Export (841)/labels/digittake-344-jpg_jpg.rf.cbe8fd13bf132dbb26a2ba0611d29c8f.txt
(5) copying: ./Image Repo/labeled/Final Roboflow Export (841)/images/screenytake-68-jpg-cropped-jpg_jpg.rf.18795a8477f8df71ca5fb0e37bfff959.jpg
(6) copying: ./Image Repo/labeled/Final Roboflow Export (841)/labels/screenytake-68-jpg-cropped-jpg_jpg.rf.18795a8477f8df71ca5fb0e37bfff959.txt
(7) copying: ./Image Repo/labeled/Final Roboflow Export (841)/images/save_dirrcropped-jpg_jpg.rf.127f02f69b5f5146c1376b9b592d58c7.jpg
(8) copy

In [ ]:
%%time
current = aw.generate_weights(3)

reading defaults from: ModelAssistedLabel config.json
reading defaults from: ModelAssistedLabel config.json
CPU times: user 1.83 ms, sys: 9.67 ms, total: 11.5 ms
Wall time: 6.41 s


In [ ]:
!ls

 00_config.ipynb
 01_split.ipynb
 02_train.ipynb
 03_detect.ipynb
'AutoWeight <name>-160068'
'AutoWeight <name>2 - 1'
'AutoWeight <name>-277648'
'AutoWeight <name>-603604'
'_capture input.ipynb'
 CONTRIBUTING.md
 data.yaml
 docker-compose.yml
 docs
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-57-37'
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-57-37.zip'
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-58-39'
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-58-39.zip'
 fromIndex-296227
'Image Repo'
 index.ipynb
 ipynb_tests
 LICENSE
 Makefile
 MANIFEST.in
 ModelAssistedLabel
'ModelAssistedLabel config.json'
'pre-trained weights'
 README.md
 settings.ini
 setup.py
 _Synch.ipynb
'test (1)'
'test (2)'
'test (3)'
'test (4)'
'test (5)'
'train (1)'
'train (2)'
'train (3)'
'train (4)'
'train (5)'
'valid (1)'
'valid (2)'
'valid (3)'
'valid (4)'
'valid (5)'
 yolov5


In [ ]:
rm -f -r 'test (1)' 'test (2)' 'test (3)' 'test (4)' 'test (5)' 'train (1)' 'train (2)' 'train (3)' 'train (4)' 'train (5)' 'valid (1)' 'valid (2)' 'valid (3)' 'valid (4)' 'valid (5)'

In [ ]:
ls

 00_config.ipynb
 01_split.ipynb
 02_train.ipynb
 03_detect.ipynb
'AutoWeight <name>-160068'/
'AutoWeight <name>2 - 1'/
'AutoWeight <name>-277648'/
'AutoWeight <name>-603604'/
'_capture input.ipynb'
 CONTRIBUTING.md
 data.yaml
 docker-compose.yml
 docs/
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-57-37'/
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-57-37.zip'
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-58-39'/
'Final Roboflow Export (841)AutoWeight <name> 21-03-21 01-58-39.zip'
 fromIndex-296227/
'Image Repo'/
 index.ipynb
 ipynb_tests/
 LICENSE
 Makefile
 MANIFEST.in
 ModelAssistedLabel/
'ModelAssistedLabel config.json'
'pre-trained weights'/
 README.md
 settings.ini
 setup.py
 _Synch.ipynb
 yolov5/


In [ ]:
!nbdev_build_lib
!nbdev_build_docs

Converted 00_config.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted 03_detect.ipynb.
Converted index.ipynb.
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/02_train.ipynb
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/01_split.ipynb


In [ ]:
!git add .

In [ ]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)

	modified:   02_train.ipynb
	modified:   yolov5 (modified content, untracked content)

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git commit -m "closing up refactor on training"

^C


In [ ]:
!git push